In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing libraries

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

# Downloads

In [ ]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

# DataSet

In [ ]:
train=pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv')
test=pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv')
sample=pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/sample_submission.csv')

# EDA and Findings

In [ ]:
train.head()

In [ ]:
train.shape

# Preprocessing

change the words to lower case

In [ ]:
train['full_text'] = train['full_text'].str.lower()
test['full_text'] = test['full_text'].str.lower()

removing stop words


In [ ]:
# Function to remove stopwords
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.lower() not in stop_words]
    return ' '.join(filtered_text)

# Apply function to 'full_text' column
train['full_text'] = train['full_text'].apply(remove_stopwords)
test['full_text'] = test['full_text'].apply(remove_stopwords)

In [ ]:
train.head()

removing all the special characters

In [ ]:
def remove_special_char(text):
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)
train["full_text"]=train["full_text"].apply(remove_special_char)
test["full_text"]=test["full_text"].apply(remove_special_char)

Lemmatization of words
 Error

In [ ]:
"""import nltk
nltk.download('wordnet')
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.lower() not in stop_words]
    
    lemmatizer = WordNetLemmatizer()
    lemmatized_text = [lemmatizer.lemmatize(word) for word in filtered_text]
    
    return ' '.join(lemmatized_text)

train['full_text'] = train['full_text'].apply(preprocess_text)
test['full_text'] = test['full_text'].apply(preprocess_text)"""

stemming

In [ ]:
from nltk.stem import PorterStemmer

def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.lower() not in stop_words]
    
    stemmer = PorterStemmer()
    stemmed_text = [stemmer.stem(word) for word in filtered_text]
    
    return ' '.join(stemmed_text)

train['full_text'] = train['full_text'].apply(preprocess_text)
test['full_text'] = test['full_text'].apply(preprocess_text)

Checking for frequent words

In [ ]:
"""from collections import Counter

# Combine all preprocessed text data into one string
text_data = ' '.join(train['full_text'].astype(str).tolist())

# Tokenize the preprocessed text (assuming each word is already stemmed and cleaned)
tokens = text_data.split()

# Count the frequency of each word
word_freq = Counter(tokens)

# Get the most common words
most_common_words = word_freq.most_common()

# Print the most common words
print(most_common_words)
"""

removing top 20% frequent words

In [ ]:
"""# Combine all preprocessed text data into one string
text_data = ' '.join(train['full_text'].astype(str).tolist())

# Tokenize the preprocessed text
tokens = text_data.split()

# Count the frequency of each word
word_freq = Counter(tokens)

# Calculate the number of top frequent words to remove (20%)
num_words_to_remove = int(len(word_freq) * 0.20)

# Get the most common words to remove
most_common_words = [word for word, freq in word_freq.most_common(num_words_to_remove)]

# Define a function to remove the most common words from a text
def remove_frequent_words(text, words_to_remove):
    return ' '.join([word for word in text.split() if word not in words_to_remove])

# Apply the function to remove the most common words from the dataset
train['full_text'] = train['full_text'].apply(lambda x: remove_frequent_words(x, most_common_words))
test['full_text'] = test['full_text'].apply(lambda x: remove_frequent_words(x, most_common_words))

# Print a sample to verify the result
print(train['full_text'].head())
print(test['full_text'].head())"""

For rare words

In [ ]:
"""from collections import Counter

# Combine all preprocessed text data into one string
text_data = ' '.join(train['full_text'].astype(str).tolist())

# Tokenize the preprocessed text
tokens = text_data.split()

# Count the frequency of each word
word_freq = Counter(tokens)

# Calculate the number of least frequent words to remove (20%)
num_words_to_remove = int(len(word_freq) * 0.20)

# Get the least common words to remove
least_common_words = [word for word, freq in word_freq.most_common()[:-num_words_to_remove-1:-1]]

# Define a function to remove the least common words from a text
def remove_rare_words(text, words_to_remove):
    return ' '.join([word for word in text.split() if word not in words_to_remove])

# Apply the function to remove the least common words from the dataset
train['full_text'] = train['full_text'].apply(lambda x: remove_rare_words(x, least_common_words))
test['full_text'] = test['full_text'].apply(lambda x: remove_rare_words(x, least_common_words))

# Print a sample to verify the result
print(train['full_text'].head())
print(test['full_text'].head())"""

In [ ]:
X = train['full_text']
y = train['score']

# Split the train data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
type(X_train) #pandas.core.series.Series
X_train

# Models

logistic regression

Accuracy: 0.47631426920854997

In [ ]:
"""pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('classifier',LogisticRegression(max_iter=1000, random_state=42))
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Predict on the validation set
y_pred = pipeline.predict(X_test)

# Calculate accuracy as a metric
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
"""
# Now, let's predict on the test data



random forest

accuracy 0.49

In [ ]:
"""
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Predict on the validation set
y_pred = pipeline.predict(X_valid)

# Calculate accuracy as a metric
accuracy = accuracy_score(y_valid, y_pred)
print("Accuracy:", accuracy)
"""

# gradient boost (best performing so far)

In [ ]:
# This the BEST CLASSIFIER SO FAR WITH TEST CLASSSIFICATON SCORE OF 0.65
# Create a pipeline with TF-IDF vectorizer and Gradient Boosting Classifier
"""pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('classifier', GradientBoostingClassifier(n_estimators=100, random_state=42))
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Predict on the validation set
y_pred = pipeline.predict(X_test)

# Calculate accuracy as a metric
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)"""


Dummy submission

In [ ]:
"""from sklearn.dummy import DummyClassifier

#Load the training data
#df = pd.read_csv("your_training_data.csv")

#Assuming 'rating' is the target variable
X = train.drop("score", axis=1)
y = train["score"]

#Create a DummyClassifier model
model = DummyClassifier(strategy="most_frequent")  # Replace 'your_strategy' with the desired strategy

#Fit the model to the training data
model.fit(X, y)

#Load the test data
#Make predictions on the test data
y_pred = model.predict(test)

#Create a submission DataFrame
submission = pd.DataFrame({"essay_id": test['essay_id'], "score": y_pred})

#Save the submission to a CSV file
submission.to_csv('submission.csv', index=False)"""

## Trying xg_boost

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
!pip install imbalanced-learn

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.feature_extraction.text import TfidfVectorizer


# Adjust class labels
y_train_for_XG = y_train - 1
y_test_for_XG = y_test - 1

# Define the pipeline with TF-IDF, SMOTE, and XGBoost classifier
pipeline = ImbPipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('smote', SMOTE(random_state=42)),
    ('xgb_classifier', xgb.XGBClassifier(objective='multi:softmax', num_class=6))
])

# Fit the training data
pipeline.fit(X_train, y_train_for_XG)

# Predict on the validation set
y_pred = pipeline.predict(X_test)

# Calculate accuracy as a metric
accuracy = accuracy_score(y_test_for_XG, y_pred)
print("Accuracy:", accuracy)


xgboost with sentence transformer

In [ ]:
"""# Install the sentence-transformers library
!pip install -q sentence-transformers

import pandas as pd
from sentence_transformers import SentenceTransformer
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Load the SentenceTransformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Generate embeddings for each essay in the training set
embeddings_train = model.encode(X_train.tolist(), convert_to_tensor=True)
embeddings_test = model.encode(X_test.tolist(), convert_to_tensor=True)

# Convert embeddings to numpy arrays
embeddings_train_np = embeddings_train.cpu().detach().numpy()
embeddings_test_np = embeddings_test.cpu().detach().numpy()

# Adjust labels for XGBoost (assuming labels start from 1)
y_train_for_XG = y_train - 1
y_test_for_XG = y_test - 1

# Define the XGBoost classifier
xgb_classifier = XGBClassifier(objective='multi:softmax', num_class=6)

# Fit the classifier on the training data
xgb_classifier.fit(embeddings_train_np, y_train_for_XG)

# Predict on the test set
y_pred = xgb_classifier.predict(embeddings_test_np)

# Calculate accuracy as a metric
accuracy = accuracy_score(y_test_for_XG, y_pred)
print("Accuracy:", accuracy)
"""

In [ ]:
unique, counts = np.unique(y_test_for_XG, return_counts=True)
dict(zip(unique, counts))

In [ ]:
test_pred = pipeline.predict(test['full_text']) + 1

# Create a submission DataFrame
submission = pd.DataFrame({"essay_id": test['essay_id'], "score": test_pred})

# Save the submission to a CSV file
submission.to_csv('submission.csv', index=False)


In [ ]:
# test_pred = pipeline.predict(test['full_text'])

# # Create a submission DataFrame
# submission = pd.DataFrame({"essay_id": test['essay_id'], "score": test_pred})

# # Save the submission to a CSV file
# submission.to_csv('submission.csv', index=False)

In [ ]:
pd.read_csv('submission.csv')